In [9]:
import os
import json
import random

# 1. 기본 설정
folder_path = 'user_profiles_split'  # JSON 파일들이 있는 폴더
locations = ['명동역', '홍대입구역', '강남역', '서울역', '서울시청', '시청역']

print(f"'{folder_path}' 폴더의 파일 처리를 시작합니다...")

# 2. 폴더 내 모든 user_*.json 파일 목록 가져오기
try:
  # user_ 로 시작하고 .json으로 끝나는 모든 파일 목록을 가져옵니다.
  all_files = [f for f in os.listdir(folder_path) if f.startswith('user_') and f.endswith('.json')]
  num_files = len(all_files)
  
  if num_files == 0:
    print(f"오류: '{folder_path}' 폴더에서 'user_*.json' 파일을 찾을 수 없습니다.")
    # 파일이 없으면 여기서 실행을 멈춥니다.
    exit()
    
  print(f"총 {num_files}개의 파일을 찾았습니다.")

except FileNotFoundError:
  print(f"오류: '{folder_path}' 폴더를 찾을 수 없습니다. 경로를 확인해주세요.")
  # 폴더가 없으면 여기서 실행을 멈춥니다.
  exit()


# 3. 위치를 "골고루" 배정하기 위한 리스트 생성
num_locations = len(locations)

# 500개 파일을 6개 장소로 나눔
base_count = num_files // num_locations  # 각 위치의 기본 할당 횟수 (500 // 6 = 83)
remainder = num_files % num_locations    # 추가로 할당해야 할 횟수 (500 % 6 = 2)

# (locations * 83) + (locations 리스트의 앞 2개)
# 이렇게 하면 83개짜리 6묶음 + 2개 = 총 500개 리스트가 생성됩니다.
# -> 4개 장소는 83번, 2개 장소는 84번 할당되어 가장 균등합니다.
distributed_locations = (locations * base_count) + locations[:remainder]

# 이 리스트를 섞어서 랜덤성을 보장합니다.
random.shuffle(distributed_locations)

# 4. 각 파일을 순회하며 내용 수정 및 저장
processed_count = 0
for i, filename in enumerate(all_files):
  # 3번에서 섞어둔 리스트에서 순서대로 위치를 하나씩 꺼냅니다.
  location_to_add = distributed_locations[i]
  file_path = os.path.join(folder_path, filename)

  try:
    # UTF-8 인코딩으로 파일 읽기
    with open(file_path, 'r', encoding='utf-8') as f:
      data = json.load(f)

    # summary_text 가져오기 (혹시 필드가 없으면 빈 문자열)
    original_text = data.get('summary_text', '')

    # 기존 텍스트의 끝 공백을 제거하고, 줄바꿈과 함께 새 위치 문장 추가
    location_sentence = f" 현재 {location_to_add} 에 있어"
    data['summary_text'] = original_text.strip() + location_sentence

    # UTF-8 인코딩 및 2칸 들여쓰기로 파일 다시 저장 (덮어쓰기)
    with open(file_path, 'w', encoding='utf-8') as f:
      json.dump(data, f, ensure_ascii=False, indent=2)
      
    processed_count += 1

  except json.JSONDecodeError:
    print(f"  [경고] {filename}: JSON 형식이 올바르지 않아 건너뜁니다.")
  except Exception as e:
    print(f"  [오류] {filename}: 처리 중 문제 발생 - {e}")

print(f"\n완료! 총 {processed_count}개의 JSON 파일에 랜덤 위치 정보를 추가했습니다.")

'user_profiles_split' 폴더의 파일 처리를 시작합니다...
총 500개의 파일을 찾았습니다.

완료! 총 500개의 JSON 파일에 랜덤 위치 정보를 추가했습니다.


In [10]:
import pandas as pd
import json
import os
import glob # 파일 목록을 검색하기 위해 import

# --- 1. 설정 ---
# (★★★) 개별 JSON 파일들이 저장된 디렉토리
PROFILE_DIR = "./user_profiles_split" 
# (★★★) 최종 저장할 CSV 파일 이름
OUTPUT_CSV = "user_profiles_combined.csv"

# --- 2. 메인 실행 함수 ---
def combine_json_files_to_csv():
  print(f"'{PROFILE_DIR}' 디렉토리에서 .json 파일 검색 중...")
  
  # PROFILE_DIR 안의 모든 .json 파일 경로를 리스트로 가져옴
  json_filepaths = glob.glob(os.path.join(PROFILE_DIR, "*.json"))
  
  if not json_filepaths:
    print(f"[오류] '{PROFILE_DIR}'에서 .json 파일을 찾을 수 없습니다.")
    print("이전 단계(preprocess_txt_to_individual_files)가 정상적으로 실행되었는지 확인하세요.")
    return

  print(f"총 {len(json_filepaths)}개의 .json 파일을 찾았습니다. DataFrame으로 변환을 시작합니다.")
  
  all_profiles_list = [] # 데이터를 저장할 리스트

  # 2. 모든 JSON 파일을 순회하며 읽기
  for filepath in json_filepaths:
    try:
      with open(filepath, 'r', encoding='utf-8') as f:
        # JSON 파일을 읽어 딕셔너리로 변환
        profile_data = json.load(f)
        all_profiles_list.append(profile_data)
        
    except Exception as e:
      print(f"[경고] {filepath} 파일 처리 중 오류 발생: {e}. 이 파일을 건너뜁니다.")
      
  if not all_profiles_list:
    print("[오류] 유효한 프로필 데이터를 하나도 읽지 못했습니다.")
    return

  # 3. 리스트를 Pandas DataFrame으로 변환
  print(f"총 {len(all_profiles_list)}개의 프로필을 DataFrame으로 변환 중...")
  df_combined = pd.DataFrame(all_profiles_list)
  
  # 4. CSV 파일로 저장
  try:
    df_combined.to_csv(OUTPUT_CSV, index=False, encoding='utf-8-sig')
    print(f"\n--- 작업 완료! ---")
    print(f"총 {len(df_combined)}개의 프로필이 '{OUTPUT_CSV}' 파일에 저장되었습니다.")
    print("\n--- CSV 파일 샘플 (상위 5개) ---")
    print(df_combined.head())
  except Exception as e:
    print(f"\n[치명적 오류] CSV 파일 저장 실패: {e}")

if __name__ == "__main__":
  combine_json_files_to_csv()

'./user_profiles_split' 디렉토리에서 .json 파일 검색 중...
총 500개의 .json 파일을 찾았습니다. DataFrame으로 변환을 시작합니다.
총 500개의 프로필을 DataFrame으로 변환 중...

--- 작업 완료! ---
총 500개의 프로필이 'user_profiles_combined.csv' 파일에 저장되었습니다.

--- CSV 파일 샘플 (상위 5개) ---
          id  ...                                       summary_text
0  user_0446  ...  안녕하세요! 저는 Giuseppe Romano입니다. 30대 이탈리아 남자로, 이번...
1  user_0016  ...  안녕하세요! 저는 Chun Hei Wong입니다. 저는 20대 남자이고, 홍콩에서 ...
2  user_0153  ...  안녕하세요! 저는 Lina Müller입니다. 독일에서 온 40대 여성이고요, 이번...
3  user_0295  ...  안녕하세요! 저는 Hazel Brown입니다. 영국에서 온 20대 여성이고요, 이번...
4  user_0041  ...  안녕하세요! 저는 Wei Zhang입니다. 저는 20대 남성이고 중국에서 왔어요. ...

[5 rows x 3 columns]


In [11]:
df_user = pd.read_csv('./user_profiles_combined.csv')
df_user

,id,name,summary_text
0,user_0446,Giuseppe Romano,"안녕하세요! 저는 Giuseppe Romano입니다. 30대 이탈리아 남자로, 이번..."
1,user_0016,Chun Hei Wong,"안녕하세요! 저는 Chun Hei Wong입니다. 저는 20대 남자이고, 홍콩에서 ..."
2,user_0153,Lina Müller,"안녕하세요! 저는 Lina Müller입니다. 독일에서 온 40대 여성이고요, 이번..."
3,user_0295,Hazel Brown,"안녕하세요! 저는 Hazel Brown입니다. 영국에서 온 20대 여성이고요, 이번..."
4,user_0041,Wei Zhang,안녕하세요! 저는 Wei Zhang입니다. 저는 20대 남성이고 중국에서 왔어요. ...
...,...,...,...
495,user_0255,Matteo Rossi,안녕하세요! 저는 Matteo Rossi입니다. 이탈리아에서 온 20대 남자예요. ...
496,user_0310,Grace Ncube,"안녕하세요! 저는 Grace Ncube입니다. 저는 30대 여자이고, 남아프리카공화..."
497,user_0081,Sebastian Müller,"안녕하세요! 저는 Sebastian Müller입니다. 저는 30대 남성이고, 독일..."
498,user_0202,Sakura Yamamoto,안녕하세요! 저는 Sakura Yamamoto입니다. 일본에서 온 20대 여자이고요...
